In [1]:
from thermo_validity import *
from ZW_model import *
from ZW_utils import *
import torch.optim as optim
import matplotlib.pyplot as plt
from ZW_Opt import *
from split_functions import one_hot_encoding, bound_creation

dataset_id = "v21D0_m1.npy"
classes = std_classes
data_split_ratio = 0.85
batch_size = 100
max_epochs = 1
learning_rate = 1e-3
block_size = 23
n_embd = 32  # 32
n_head = 4  # 4
n_layer = 2  # 2
dropout = 0.1  # 0.1
vocab_size = len(classes)
model = GPT(vocab_size, n_embd, n_head, n_layer, block_size, dropout)
loss_function = std_loss
augmentation = False
N1 = 10
cycles1 = 11
N2 = 3
cycles2 = 8
cutoff = 143.957



def Transformer_training_cycle(mode, N, save_path, dataset, cycles):
    if mode == "M1":
        for i in range(0, cycles):
            train_loader, val_loader = T_integer_data_loaders(
                dataset, batch_size, data_split_ratio, classes, block_size, augmentation
            )
            model = GPT(vocab_size, n_embd, n_head, n_layer, block_size, dropout)
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)
            best_model, last_model, t_loss, v_loss = T_integer_training(
                model, optimizer, loss_function, train_loader, val_loader, max_epochs
            )
            plot_name = mode + "_loss_" + str(i) + ".png"
            model_name = mode + "_model_" + str(i) + ".pt"
            data_name = mode + "_data_" + str(i + 1) + ".npy"
            plt.plot(t_loss, label="Training Loss")
            plt.plot(v_loss, label="Validation Loss")
            plt.legend()
            plt.savefig(f"{save_path}/{plot_name}")
            plt.clf()
            torch.save(best_model, f"{save_path}/{model_name}")
            model.load_state_dict(best_model)
            layout_list = transformer_generation(model, classes, N)
            np.save(f"{save_path}/{mode}generated_{i}.npy", layout_list)
            new_strings = np.array(validity(layout_list), dtype=object)
            prev_strings = np.array(dataset, dtype=object)
            new_dataset = np.unique(np.concatenate((prev_strings, new_strings), axis=0))
            np.save(f"{save_path}/{data_name}", new_dataset)
            dataset = new_dataset.tolist()
            print("Dataset Length:", len(dataset))
    else:
        for i in range(0, cycles):
            train_loader, val_loader = T_integer_data_loaders(
                dataset, batch_size, data_split_ratio, classes, block_size, augmentation
            )
            model = GPT(vocab_size, n_embd, n_head, n_layer, block_size, dropout)
            model.load_state_dict(torch.load(f"{save_path}/M1_model_10.pt"))
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)
            best_model, last_model, t_loss, v_loss = T_integer_training(
                model, optimizer, loss_function, train_loader, val_loader, max_epochs
            )
            plot_name = mode + "_loss_" + str(i) + ".png"
            model_name = mode + "_model_" + str(i) + ".pt"
            data_name = mode + "_data_" + str(i + 1) + ".npy"
            plt.plot(t_loss, label="Training Loss")
            plt.plot(v_loss, label="Validation Loss")
            plt.legend()
            plt.savefig(f"{save_path}/{plot_name}")
            plt.clf()
            torch.save(best_model, f"{save_path}/{model_name}")
            model.load_state_dict(best_model)
            layout_list = transformer_generation(model, classes, N)
            np.save(f"{save_path}/{mode}generated_{i}.npy", layout_list)
            unique_strings = np.unique(np.array(validity(datalist), dtype=object))
            p_datalist = dataset
            datalist = np.unique(np.concatenate((p_datalist, unique_strings), axis=0))
            candidates = datalist[
                np.where(np.isin(unique_strings, p_datalist, invert=True))[0]
            ]
            candidates_results = optimization(
                candidates, classes, save_path, "candidates_" + str(i)
            )
            good_layouts = optimization_filter(candidates_results, candidates, cutoff)
            dataset = np.unique(np.concatenate((p_datalist, good_layouts), axis=0))
            np.save(f"{save_path}/{data_name}", dataset)
    return model


def optimization(data_array, classes, save_path, save_name):
    one_hot_tensors = np.array(one_hot_encoding(data_array, classes), dtype=object)
    print(one_hot_tensors.shape)
    valid_layouts = set()
    penalty_layouts = set()
    broken_layouts = set()
    results = np.zeros(one_hot_tensors.shape[0])
    positions = np.zeros(one_hot_tensors.shape[0], dtype=object)
    for i in range(one_hot_tensors.shape[0]):
        layout = one_hot_tensors[i]
        equipment, bounds, x, splitter = bound_creation(layout)
        # PSO Parameters
        swarmsize_factor = 7
        particle_size = swarmsize_factor * len(bounds)
        if 5 in equipment:
            particle_size += -1 * swarmsize_factor
        if 9 in equipment:
            particle_size += -2 * swarmsize_factor
        iterations = 30
        nv = len(bounds)
        try:
            a = PSO(
                objective_function, bounds, particle_size, iterations, nv, equipment
            )
            if a.result < 1e6:
                valid_layouts.add(i)
                results[i] = a.result
                positions[i] = a.points
            else:
                penalty_layouts.add(i)
        except:
            broken_layouts.add(i)
        if i % 100 == 0:
            print(
                "Valid/Penalty/Broken",
                len(valid_layouts),
                len(penalty_layouts),
                len(broken_layouts),
            )
    results_name = "results_" + save_name + ".npy"
    positions_name = "positions_" + save_name + ".npy"
    np.save(f"{save_path}\{results_name}", results)
    np.save(f"{save_path}\{positions_name}", positions)
    return results


def optimization_filter(results, datalist, cutoff, save_name):
    nonzero_results = results[np.where(results > 0)]
    good_layouts = []
    good_results = []
    print("Optimization Results:", len(nonzero_results), len(results))
    for i in range(len(results)):
        if results[i] < cutoff and results[i] > 0:
            good_layouts.append(datalist[i])
            good_results.append(results[i])
    print("Good layouts", len(good_layouts))
    good_layouts = np.array(good_layouts, dtype=object)
    np.save(
        f"{save_path}/{save_name}_good_layouts.npy",
        good_layouts,
    )
    return good_layouts, good_results


In [5]:
# save_path = make_dir(
#     model,
#     batch_size,
#     learning_rate,
# )
dataset = dataloading(dataset_id)

save_path = "202407191642_GPT_batch100_lr1e-03"


# dataset = np.load(f"{save_path}/generated+1_data.npy", allow_pickle=True).tolist()


Designs in the dataset v21D0_m1.npy : 1000


Designs in the dataset: 1000
Input shape: torch.Size([1000, 22])
Training set size: 850


IndexError: index out of range in self